In [ ]:
!pip install -q sentence-transformers pandas scikit-learn

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error
from scipy.stats import spearmanr, pearsonr

In [ ]:
DATA_PATH = '/content/drive/MyDrive/Datasets/cleaned_dataset.csv'

In [ ]:
df = pd.read_csv(DATA_PATH)
df = df.sample(3000, random_state=42)
df = df.dropna(subset=["text1", "text2", "label"])
train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

train_samples = [
    InputExample(texts=[row['text1'], row['text2']], label=float(row['label']))
    for _, row in train_df.iterrows()
]

val_pairs = list(zip(val_df['text1'].tolist(), val_df['text2'].tolist()))
val_labels = val_df['label'].astype(float).tolist()

In [ ]:
MODEL_NAME = 'distiluse-base-multilingual-cased-v1'
EPOCHS = 1
BATCH_SIZE = 128
SAVE_DIR = '/content/fine-tuned-model'

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"📦 Using device: {device}")

model = SentenceTransformer(MODEL_NAME, device=device)
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=BATCH_SIZE)
train_loss = losses.CosineSimilarityLoss(model=model)

📦 Using device: cuda


In [ ]:
def evaluate_model(model, pairs, true_labels):
    embeddings1 = model.encode([p[0] for p in pairs], convert_to_tensor=True, show_progress_bar=False)
    embeddings2 = model.encode([p[1] for p in pairs], convert_to_tensor=True, show_progress_bar=False)
    cosine_scores = torch.nn.functional.cosine_similarity(embeddings1, embeddings2).cpu().numpy()

    roc = roc_auc_score(true_labels, cosine_scores)
    mse = mean_squared_error(true_labels, cosine_scores)
    spearman = spearmanr(true_labels, cosine_scores).correlation
    pearson = pearsonr(true_labels, cosine_scores)[0]

    print(f"\n🔍 Validation Metrics:")
    print(f"ROC AUC:    {roc:.4f}")
    print(f"MSE:        {mse:.4f}")
    print(f"Spearman:   {spearman:.4f}")
    print(f"Pearson:    {pearson:.4f}")
    return roc

In [ ]:
warmup_steps = int(len(train_dataloader) * EPOCHS * 0.1)
best_roc_auc = 0

for epoch in range(EPOCHS):
    print(f"\n🚀 Epoch {epoch + 1}/{EPOCHS}")
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        warmup_steps=warmup_steps,
        show_progress_bar=True
    )

    roc_auc = evaluate_model(model, val_pairs, val_labels)

    if roc_auc > best_roc_auc:
        best_roc_auc = roc_auc
        model.save(SAVE_DIR)
        print(f"✅ Model saved to {SAVE_DIR}")


🚀 Epoch 1/1


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss



🔍 Validation Metrics:
ROC AUC:    1.0000
MSE:        0.0099
Spearman:   0.0816
Pearson:    0.2380
✅ Model saved to /content/fine-tuned-model


In [ ]:
import shutil
from google.colab import files

shutil.make_archive("fine-tuned-model", 'zip', SAVE_DIR)
files.download("fine-tuned-model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>